## Libraries

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/kevinscaria/InstructABSA.git

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
import os
import torch

if IN_COLAB:
    root_path = '/content/InstructABSA'
else:
    root_path = 'Enter local path'

use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

In [ ]:
task_name = 'joint_task'
experiment_name = 'accomm_absa'
model_checkpoint = 'kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined'
print('Experiment Name: ', experiment_name)
model_out_path = '/content/drive/MyDrive/Model/joint_task/kevinscariajoint_tk-instruct-base-def-pos-neg-neut-combined-accomm_absa'
# model_out_path = './Models'
# model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
# print('Model output path: ', model_out_path)

## Training

In [ ]:
import pandas as pd
import ast
from InstructABSA.utils import T5Generator
from InstructABSA.data_prep import DatasetLoader
from instructions import InstructionsHandler

# 1. Load data
id_train_file_path = '/content/accomm_train_0529_concat_anchae.csv'
id_test_file_path = '/content/accomm_test_0529_concat_anchae.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# 2. Create labels in term(category):polarity format
def format_labels(row):
    try:
        aspects = ast.literal_eval(row) if isinstance(row, str) else row
    except:
        return 'noaspect:none'

    if not aspects:
        return 'noaspect:none'

    labels = [
        f"{item['term']}.{item['category']}:{item['polarity']}"
        for item in aspects
    ]

    return ', '.join(labels)

id_tr_df['labels'] = id_tr_df['aspectTerms'].apply(format_labels)
id_te_df['labels'] = id_te_df['aspectTerms'].apply(format_labels)

bos_instruction =
"""Definition: The output will be the aspect terms, their predefined categories, and sentiment polarity.
Format each as term.category:polarity. The category must be one of the following:
Cleanliness, Communication, Location, Accuracy, Check-in, Amenity, or Value.
In cases where no aspect exists and no category exists, output should be noaspectterm.noaspectcategory:none.

Positive example 1-
input: The room was clean and I slept very well.
output: room.Cleanliness:positive

Positive example 2-
input: The host was super responsive and they provide new TV.
output: host.Communication:positive, TV.Amenity:positive

Negative example 1-
input: The place was dirty and check-in was delayed.
output: place.Cleanliness:negative, check-in.Check-in:negative

Negative example 2-
input: The listing photos were inaccurate.
output: photos.Accuracy:negative

Neutral example 1-
input: The price was reasonable for a one-night stay.
output: price.Value:neutral

Neutral example 2-
input: It would take around 10 to 15 mins walk to the subway station.
output: subway station.Location:neutral

Now complete the following example-
input: """

eos_instruction = "\noutput:"

id_tr_df['text'] = id_tr_df['raw_text'].apply(lambda x: bos_instruction + x + eos_instruction)
id_te_df['text'] = id_te_df['raw_text'].apply(lambda x: bos_instruction + x + eos_instruction)

In [ ]:
id_tr_df[id_tr_df['labels'] == 'noaspect:none']

,raw_text,aspectTerms,labels,text


In [ ]:
id_te_df[id_te_df['labels'] == 'noaspect:none']

,raw_text,aspectTerms,labels,text


In [ ]:
id_tr_df.iloc[0]

,0
raw_text,"The apartment was very nice, quite new and clean."
aspectTerms,"[{'term': 'apartment', 'category': 'Location',..."
labels,"apartment.Location:positive, apartment.Cleanli..."
text,Definition: The output will be the aspect term...


In [ ]:
loader = DatasetLoader(id_tr_df[['text', 'labels']], id_te_df[['text', 'labels']])
# 먼저 T5Generator 객체를 생성
t5_exp = T5Generator(model_checkpoint)

# 인스턴스의 토크나이즈 함수 넘기기
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':16,
    'num_train_epochs':8,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'load_best_model_at_end':False,
    'logging_strategy': 'epoch',
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps,
    'report_to': 'none'
}

model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

tokenizer_config.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

...

Map:   0%|          | 0/704 [00:00<?, ? examples/s]

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Trainer device: cuda:0

Model training started ....


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
88,0.764300
176,0.284700
264,0.210200
352,0.174200
440,0.142700
528,0.126300
616,0.117700
704,0.119700


## Inference

In [ ]:
# loader = DatasetLoader(id_tr_df[['text', 'labels']], id_te_df[['text', 'labels']])
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

Map:   0%|          | 0/704 [00:00<?, ? examples/s]

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Model loaded to:  cuda


100%|██████████| 44/44 [00:53<00:00,  1.22s/it]


Model loaded to:  cuda


100%|██████████| 19/19 [00:23<00:00,  1.25s/it]


In [ ]:
# chae
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.813953488372093
Train Recall:  0.8309591642924976
Train F1:  0.8223684210526315
Test Precision:  0.6190476190476191
Test Recall:  0.6635730858468677
Test F1:  0.6405375139977603


In [ ]:
import shutil
from google.colab import files

# 1. zip으로 압축
shutil.make_archive('model_checkpoint', 'zip', model_out_path)

# 2. 다운로드
files.download('model_checkpoint.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

실제 결과

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = model_out_path  # 학습된 모델이 저장된 폴더
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

bos_instruction = """Definition: The output will be the aspect terms, their predefined categories, and sentiment polarity. Format each as term.category:polarity. The category must be one of the following: Cleanliness, Communication, Location, Accuracy, Check-in, Amenity, or Value. In cases where no aspect exists and no category exists, output should be noaspectterm(noaspectcategory):none.

Positive example 1-
input: The room was clean and I slept very well.
output: room.Cleanliness:positive

Positive example 2-
input: The host was super responsive and they provide new TV.
output: host.Communication:positive, TV.Amenity:positive

Negative example 1-
input: The place was dirty and check-in was delayed.
output: place.Cleanliness:negative, check-in.Check-in:negative

Negative example 2-
input: The listing photos were inaccurate.
output: photos.Accuracy:negative

Neutral example 1-
input: The price was reasonable for a one-night stay.
output: price.Value:neutral

Neutral example 2-
input: It would take around 10 to 15 mins walk to the subway station.
output: subway station.Location:neutral

Now complete the following example-
input: """
delim_instruct = ''
eos_instruct = ' \noutput:'
text = "location is very great and there is a lots of amenities, value for the money"

tokenized_text = tokenizer(bos_instruction + text + delim_instruct + eos_instruct, return_tensors="pt")
output = model.generate(tokenized_text.input_ids, max_length=128)
print('Model output: ', tokenizer.decode(output[0], skip_special_tokens=True))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Model output:  location.Location:positive, amenities.Amenity:positive, value.Value:positive


## loss로 score 추론

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from torch.nn.functional import softmax
import torch.nn.functional as F

model_path = model_out_path  # 학습된 모델이 저장된 폴더
# model = T5ForConditionalGeneration.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_path)
model.eval()


# 2. 기본 instruction prompt 정의
def build_prompt(input_text):
    instruction = """Definition: The output will be the aspect terms, their predefined categories, and sentiment polarity. Format each as term.category:polarity. The category must be one of the following: Cleanliness, Communication, Location, Accuracy, Check-in, Amenity, or Value. In cases where no aspect exists and no category exists, output should be noaspectterm(noaspectcategory):none.

Positive example 1-
input: The room was clean and I slept very well.
output: room.Cleanliness:positive

Positive example 2-
input: The host was super responsive and they provide new TV.
output: host.Communication:positive, TV.Amenity:positive

Negative example 1-
input: The place was dirty and check-in was delayed.
output: place.Cleanliness:negative, check-in.Check-in:negative

Negative example 2-
input: The listing photos were inaccurate.
output: photos.Accuracy:negative

Neutral example 1-
input: The price was reasonable for a one-night stay.
output: price.Value:neutral

Neutral example 2-
input: It would take around 10 to 15 mins walk to the subway station.
output: subway station.Location:neutral

Now complete the following example-
input: """ + input_text + "\noutput:"
    return instruction

# 3. 감성 확률 추정 함수
def get_sentiment_probs(prompt_text, aspect_term):
    sentiments = ["positive", "neutral", "negative"]
    losses = []
    for sentiment in sentiments:
        output_seq = f"{aspect_term}:{sentiment}"
        inputs = tokenizer(prompt_text, return_tensors="pt").to(device)
        labels = tokenizer(output_seq, return_tensors="pt").input_ids.to(device)
        labels[labels == tokenizer.pad_token_id] = -100

        with torch.no_grad():
            output = model(**inputs, labels=labels)
            loss = output.loss.item()
            losses.append(-loss)  # log-likelihood

    probs = F.softmax(torch.tensor(losses), dim=0)
    return dict(zip(sentiments, [round(p.item(), 4) for p in probs]))

# 4. 모델 예측 및 전체 파이프라인
def analyze_aspects(input_sentence):
    prompt = build_prompt(input_sentence)

    # 모델 output 생성
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=128)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "noaspectterm.noaspectcategory:none" in output_text.lower():
        return {}

    # term.category:polarity 파싱
    aspect_terms = []
    for pair in output_text.split(","):
        if ":" in pair and "." in pair:
            try:
                term_cat, polarity = pair.strip().split(":")
                term, category = term_cat.strip().split(".")
                aspect_terms.append((term.strip(), category.strip(), polarity.strip()))
            except ValueError:
                continue  # 잘못된 형식은 무시
        else:
            continue  # 필수 구분자 없으면 무시

    # 감성 확률 계산
    result = {}
    for term, category, _ in aspect_terms:
        aspect_string = f"{term}.{category}"  # 예: host.Communication
        probs = get_sentiment_probs(prompt, aspect_string)
        score = convert_sentiment_to_score(probs)
        result[aspect_string] = {
            "probs": probs,
            "score": score
        }

    return result

def convert_sentiment_to_score(sentiment_probs: dict) -> float:
    """
    감성 확률을 기반으로 0~5점 점수 계산
    """
    weights = {
        "positive": 6.0,
        "neutral": 3.0,
        "negative": 1.0
    }
    score = sum(weights[s] * sentiment_probs[s] for s in sentiment_probs)
    return round(score, 2)




## 추론 예시

In [ ]:
# input_sentence = "The location is not good but host is responsive"
# result = analyze_aspects(input_sentence)

# from pprint import pprint
# pprint(result)

{'host.Communication': {'probs': {'negative': 0.3268,
                                  'neutral': 0.3028,
                                  'positive': 0.3703},
                        'score': 3.46},
 'location.Location': {'probs': {'negative': 0.4888,
                                 'neutral': 0.2926,
                                 'positive': 0.2186},
                       'score': 2.68}}


In [ ]:
input_sentence = "The location is terrible but host is responsive and room is clean"
result = analyze_aspects(input_sentence)

pprint(result)

{'host.Communication': {'probs': {'negative': 0.2356,
                                  'neutral': 0.2906,
                                  'positive': 0.4738},
                        'score': 3.95},
 'location.Location': {'probs': {'negative': 0.5055,
                                 'neutral': 0.2695,
                                 'positive': 0.225},
                       'score': 2.66},
 'room.Cleanliness': {'probs': {'negative': 0.2176,
                                'neutral': 0.3116,
                                'positive': 0.4708},
                      'score': 3.98}}


In [ ]:
input_sentence = "The location is not close to station but host is responsive and room is clean"
result = analyze_aspects(input_sentence)

pprint(result)

{'host.Communication': {'probs': {'negative': 0.1908,
                                  'neutral': 0.2788,
                                  'positive': 0.5304},
                        'score': 4.21},
 'location.Location': {'probs': {'negative': 0.4443,
                                 'neutral': 0.3225,
                                 'positive': 0.2332},
                       'score': 2.81},
 'room.Cleanliness': {'probs': {'negative': 0.1951,
                                'neutral': 0.3182,
                                'positive': 0.4868},
                      'score': 4.07},
 'station.Location': {'probs': {'negative': 0.4218,
                                'neutral': 0.3578,
                                'positive': 0.2205},
                      'score': 2.82}}


In [ ]:
input_sentence = "The accommodation is far from station but host is responsive and room is clean"
result = analyze_aspects(input_sentence)

pprint(result)

{'accommodation.Location': {'probs': {'negative': 0.4199,
                                      'neutral': 0.3286,
                                      'positive': 0.2515},
                            'score': 2.91},
 'host.Communication': {'probs': {'negative': 0.1839,
                                  'neutral': 0.2724,
                                  'positive': 0.5437},
                        'score': 4.26},
 'room.Cleanliness': {'probs': {'negative': 0.1969,
                                'neutral': 0.3277,
                                'positive': 0.4754},
                      'score': 4.03},
 'station.Location': {'probs': {'negative': 0.4122,
                                'neutral': 0.3562,
                                'positive': 0.2316},
                      'score': 2.87}}


In [ ]:
input_sentence = "The accommodation is really far from station but host is responsive and room is very clean"
result = analyze_aspects(input_sentence)

pprint(result)

{'accommodation.Location': {'probs': {'negative': 0.4404,
                                      'neutral': 0.3069,
                                      'positive': 0.2527},
                            'score': 2.88},
 'host.Communication': {'probs': {'negative': 0.2009,
                                  'neutral': 0.2772,
                                  'positive': 0.5219},
                        'score': 4.16},
 'room.Cleanliness': {'probs': {'negative': 0.2022,
                                'neutral': 0.3163,
                                'positive': 0.4815},
                      'score': 4.04},
 'station.Location': {'probs': {'negative': 0.425,
                                'neutral': 0.3418,
                                'positive': 0.2332},
                      'score': 2.85}}


In [ ]:
input_sentence = "The location is terrible but host is responsive"
result = analyze_aspects(input_sentence)

pprint(result)


📌 Model Output: location.Location:negative, host.Communication:positive
{'host.Communication': {'probs': {'negative': 0.2464,
                                  'neutral': 0.2449,
                                  'positive': 0.5087},
                        'score': 4.03},
 'location.Location': {'probs': {'negative': 0.527,
                                 'neutral': 0.251,
                                 'positive': 0.2219},
                       'score': 2.61}}


In [ ]:
input_sentence = "The location is terrible but host is responsive"
result = analyze_aspects(input_sentence)

pprint(result)

{'host.Communication': {'probs': {'negative': 0.3049,
                                  'neutral': 0.2767,
                                  'positive': 0.4184},
                        'score': 3.65},
 'location.Location': {'probs': {'negative': 0.5227,
                                 'neutral': 0.2569,
                                 'positive': 0.2204},
                       'score': 2.62}}


In [ ]:
input_sentence = "The location is terrible but host is really responsive"
result = analyze_aspects(input_sentence)

pprint(result)

{'host.Communication': {'probs': {'negative': 0.2351,
                                  'neutral': 0.2647,
                                  'positive': 0.5002},
                        'score': 4.03},
 'location.Location': {'probs': {'negative': 0.5177,
                                 'neutral': 0.2618,
                                 'positive': 0.2205},
                       'score': 2.63}}
